# FP-growth算法
## FP树
一颗FP树看上去与计算机科学中的其他树结构类似，但是它通过链接（link）来连接相似元素，被连起来的元素项可以看成一个链表。

同搜索树不同的是，一个元素项可以在一棵FP树上出现多次。FP树会存储项集的出现频率，而每个项集都会以路径的方式存储在树中。
## 构建FP树

In [1]:
class TreeNode:
    def __init__(self, name, n_occur, parent):
        self.name = name
        self.count = n_occur
        self.node_link = None
        self.parent = parent
        self.children = {}
    
    def increase(self, n_occur):
        self.count += n_occur
        
    def dispose(self, ind=1):
        print(' ' * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.dispose(ind + 1)

In [3]:
root = TreeNode('one', 13, None)
root.dispose()

  one   13


In [4]:
root.children['two'] = TreeNode('two', 9, None)
root.dispose()

  one   13
   two   9


In [5]:
root.children['three'] = TreeNode('three', 32, None)
root.dispose()

  one   13
   two   9
   three   32


还需要一个头指针表来指向给定类型的第一个实例。利用头指针表，可以快速访问FP树中一个给定类型的所有元素。

最后生成的FP树如下所示：

![FP-tree](FP-tree.png)

算法流程：
1. 遍历所有的数据集合，计算所有项的支持度。
2. 丢弃非频繁的项
3. 基于支持度降序排序所有的项（头指针表的顺序）
4. 所有数据集合按照得到的顺序重新整理
5. 整理完成后，丢弃每个集合末尾的非频繁项
6. 读取每个集合插入FP树中，同时用一个头部链表数据结构维护不同集合的相同项

In [6]:
def load_dataset():
    dataset = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return dataset

In [7]:
def create_init_set(dataset):
    ret_dict = {}
    for trans in dataset:
        ret_dict[frozenset(trans)] = 1  # frozenset是不可变的集合
    return ret_dict

In [8]:
dataset = load_dataset()
dataset

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [9]:
init_set = create_init_set(dataset)
init_set

{frozenset({'z'}): 1,
 frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [10]:
def updata_header(node_to_test, target_node):
    """每次增加新的树节点，都要在头指针表的相应链表最后追加新的链表节点"""
    while node_to_test.node_link is not None:  # Do not use recursion to traverse a linked list!
        node_to_test = node_to_test.node_link
    node_to_test.node_link = target_node

In [11]:
def updata_tree(items, tree, header_table, count):
    """
    为FP树添加新的节点
    :param items: 有序频繁项集
    :param tree:
    :param header_table:
    :param count: 频繁项集整体的频数
    :return:
    """
    if items[0] in tree.children:  # check if orderedItems[0] in retTree.children
        tree.children[items[0]].increase(count)  # incrament count
    else:  # add items[0] to inTree.children
        tree.children[items[0]] = TreeNode(items[0], count, tree)
        if header_table[items[0]][1] is None:  # update header table
            header_table[items[0]][1] = tree.children[items[0]]
        else:
            updata_header(header_table[items[0]][1], tree.children[items[0]])
    if len(items) > 1:  # call updateTree() with remaining ordered items
        updata_tree(items[1::], tree.children[items[0]], header_table, count)

In [12]:
def create_tree(dataset, min_support=1):
    """
    create FP-tree from dataset but don't mine
    :param dataset: dict, key: 频繁项集, value: 出现次数
    :param min_support: 最小支持度
    :return:
    """
    header_table = {}

    '''first pass counts frequency of occurance'''
    for transaction in dataset:
        for item in transaction:
            header_table[item] = header_table.get(item, 0) + dataset[transaction]

    '''remove items not meeting minSup'''
    new_header_table = header_table.copy()
    for k in header_table.keys():
        if header_table[k] < min_support:
            del (new_header_table[k])

    freq_item_set = set(new_header_table.keys())  # 获取频繁项集
    if len(freq_item_set) == 0:
        return None, None

    '''reformat headerTable to use Node link'''
    for k in new_header_table:
        new_header_table[k] = [new_header_table[k], None]  # [5表示出现次数，[树节点]]

    tree = TreeNode('Null Set', 1, None)
    for tran_set, count in dataset.items():
        local_d = {}  # 当前频繁项集的频数统计字典
        for item in tran_set:  # put transaction items in order
            if item in freq_item_set:
                local_d[item] = new_header_table[item][0]
        if len(local_d) > 0:
            ordered_items = [v[0] for v in sorted(local_d.items(), key=lambda p: p[1], reverse=True)]
            updata_tree(ordered_items, tree, new_header_table, count)  # populate tree with ordered freq itemset
    return tree, new_header_table

In [20]:
mytree, myheader_table = create_tree(init_set, min_support=3)

In [21]:
mytree.dispose()

  Null Set   1
   z   5
    r   1
    x   3
     t   2
      s   2
       y   2
     r   1
      t   1
       y   1
   x   1
    r   1
     s   1


In [22]:
myheader_table

{'r': [3, <__main__.TreeNode at 0x229e1e31a90>],
 's': [3, <__main__.TreeNode at 0x229e1e3e860>],
 't': [3, <__main__.TreeNode at 0x229e1e3e1d0>],
 'x': [4, <__main__.TreeNode at 0x229e1e31b00>],
 'y': [3, <__main__.TreeNode at 0x229e1e3e710>],
 'z': [5, <__main__.TreeNode at 0x229e1e31780>]}

## 从一棵FP树中挖掘频繁项集
算法流程：
1. 从FP树中获得条件模式基
2. 利用条件模式基，构建一个条件FP树
3. 迭代重复步骤1和2，知道树包含一个元素项为止

条件模式基：首先从头指针表中的单个频繁元素项开始，对于每一个元素项，获得其对应的条件模式基。条件模式基是以所查找的元素项为结尾的路径集合。

找到条件模式基的示意图如下：
![condition_patterns](condition_patterns.png)

In [16]:
def ascend_tree(leaf_node, prefix_path):
    """ascends from leaf node to root"""
    if leaf_node.parent is not None:
        prefix_path.append(leaf_node.name)
        ascend_tree(leaf_node.parent, prefix_path)

In [17]:
def find_prefix_path(tree_node):
    """treeNode comes from header table"""
    condition_patterns = {}
    while tree_node is not None:
        prefix_path = []
        ascend_tree(tree_node, prefix_path)
        if len(prefix_path) > 1:
            condition_patterns[frozenset(prefix_path[1:])] = tree_node.count
        tree_node = tree_node.node_link
    return condition_patterns

In [24]:
find_prefix_path(myheader_table['y'][1])

{frozenset({'s', 't', 'x', 'z'}): 2, frozenset({'r', 't', 'x', 'z'}): 1}

### 条件数与频繁项集
按频数从小到大的顺序，对每一个元素项都要计算条件模式基；

然后以条件模式基为新数据集构造条件FP树，条件FP树如下图所示：
![condition_tree](condition_tree.png)

对条件FP树，从叶节点回溯到根节点，得到频繁项集。

In [25]:
def mine_tree(header_table, min_support, pre_fix, freq_item_list):
    """
    挖掘频繁项
    :param header_table: 头指针表
    :param min_support: 最小支持度
    :param pre_fix:
    :param freq_item_list: 频繁项集
    :return:
    """
    resorted_header_table = [v[0] for v in sorted(header_table.items(), key=lambda p: p[1][0])]

    for base_pattern in resorted_header_table:  # start from bottom of header table
        # print(base_pattern)
        new_freq_set = pre_fix.copy()
        new_freq_set.add(base_pattern)
        # print('finalFrequent Item: ', new_freq_set)  # append to set

        freq_item_list.append(new_freq_set)
        condition_patterns = find_prefix_path(header_table[base_pattern][1])
        # print('condPattBases :', base_pattern, condition_patterns)
        # 2. construct cond FP-tree from cond. pattern base
        my_cond_tree, my_head = create_tree(condition_patterns, min_support)
        # print('head from conditional tree: ', my_head)
        if my_head is not None:  # 3. mine cond. FP-tree
#             print('conditional tree for: ', new_freq_set)
            my_cond_tree.dispose(1)
            mine_tree(my_head, min_support, new_freq_set, freq_item_list)

In [26]:
myfreq_item_list = []
mine_tree(myheader_table, 3, set([]), myfreq_item_list)

  Null Set   1
   x   3
    z   3
  Null Set   1
   x   3
  Null Set   1
   x   3
  Null Set   1
   x   3
    t   3
     z   3
  Null Set   1
   x   3
  Null Set   1
   x   3
    t   3
  Null Set   1
   x   3
  Null Set   1
   z   3


In [27]:
myfreq_item_list

[{'r'},
 {'t'},
 {'t', 'x'},
 {'t', 'z'},
 {'t', 'x', 'z'},
 {'s'},
 {'s', 'x'},
 {'y'},
 {'x', 'y'},
 {'t', 'y'},
 {'t', 'x', 'y'},
 {'y', 'z'},
 {'x', 'y', 'z'},
 {'t', 'y', 'z'},
 {'t', 'x', 'y', 'z'},
 {'x'},
 {'x', 'z'},
 {'z'}]